In [8]:
import pandas as pd
import numpy as np

basedir="/cbscratch/franco/datasets/FHS/"
c2_labfile=basedir+"phs000007.v30.pht002889.v3.p11.c2.l_fhslab_2011_m_0656s.HMB-IRB-NPU-MDS.txt.gz"
c1_labfile=basedir+"phs000007.v30.pht002889.v3.p11.c1.l_fhslab_2011_m_0656s.HMB-IRB-MDS.txt.gz"
c2_agefile=basedir+"phs000007.v30.pht000740.v8.p11.c2.birthyr_alls.HMB-IRB-NPU-MDS.txt.gz"
c1_agefile=basedir+"phs000007.v30.pht000740.v8.p11.c1.birthyr_alls.HMB-IRB-MDS.txt.gz"

# Load pedigree family data
pedigree_file=basedir+"phs000007.v30.pht000183.v13.p11.Framingham_Pedigree.MULTI.txt.gz"
ped_df = pd.read_csv(pedigree_file, comment="#", sep="\t", header=0)
print("pedigree:", ped_df.shape)
# ped_df.groupby("pedno").size()


agec1 = pd.read_csv(c1_agefile, comment="#", sep="\t", header=0)
agec2 = pd.read_csv(c2_agefile, comment="#", sep="\t", header=0)
ages_df = pd.concat([agec1, agec2])

# approximate age of the samples.. although I don't know when they were actually collected
ages_df["age"] = 2010 - ages_df.birthyr

print("ages",ages_df.shape) 
ped_ages_df = pd.merge(ped_df, ages_df, on=["dbGaP_Subject_ID", "shareid", "idtype"] )
print(ped_ages_df.shape) # 12680, 12
                                  

pedigree: (15882, 8)
ages (14154, 5)
(12680, 10)


In [10]:
# #################################
# Concatenate expression master files for both consent groups
# #################################

# Expression Master files
mf3 = basedir+"expression/MasterFile_Gene_GENIII_3180.txt"
mfoff = basedir+"expression/MasterFile_Gene_OFF_2446.txt"

df_gen3 = pd.read_csv(mf3, sep="\t", header=0)
df_off = pd.read_csv(mfoff, sep="\t", header=0)

print("gen3",df_gen3.shape)
print("off",df_off.shape)

probe_columns = ["cel_files", \
                 "RevisedSabreID", \
                 "all_probeset_mean", \
                 "all_probeset_stdev", \
                 "neg_control_mean", \
                 "neg_control_stdev", \
                 "pos_control_mean",\
                 "pos_control_stdev", \
                 "all_probeset_rle_mean", \
                 "all_probeset_mad_residual_mean", \
                 "mm_mean"]

expr_data_df = pd.concat([ df_off[probe_columns], df_gen3[probe_columns] ])
print(expr_data_df.shape)

# # Some checks for RNA integrity, etc.. not really used in any publication that I've seen
# df_gen3.iloc[:,86:88].groupby("As_RNA Integrity Status").count()
# df_gen3.iloc[:,86:88].groupby("As_RNA Purity Status").count()

gen3 (3180, 121)
off (2446, 105)
(5626, 11)


In [11]:
#######################################
# Compile Sample data and merge parent and child studies IDs
#######################################

## Get Sample Info from parent and child study
fhs_samples_file = basedir+"phs000007.v30.pht001415.v18.p11.Framingham_Sample.MULTI.txt.gz"
expr_samples_c1 = basedir+"expression/phs000363.v17.pht002944.v4.p11.c1.Framingham_SABRe_Sample_Attributes.HMB-IRB-MDS.txt.gz"
expr_samples_c2 = basedir+"expression/phs000363.v17.pht002944.v4.p11.c2.Framingham_SABRe_Sample_Attributes.HMB-IRB-NPU-MDS.txt.gz"

# Read in and merge both consent groups
fhs_samples_df = pd.read_csv(fhs_samples_file, comment="#", sep="\t", header=0)
expr_samples_c1_df = pd.read_csv(expr_samples_c1, comment="#", sep="\t", header=0)
expr_samples_c2_df = pd.read_csv(expr_samples_c2, comment="#", sep="\t", header=0)
expr_samples_df = pd.concat( [expr_samples_c1_df, expr_samples_c2_df])

# merge parent and child studies
print("expr_samples_df", expr_samples_df.shape)
print("fhs_samples_df", fhs_samples_df.shape)
fhs_expr_samples = pd.merge(expr_samples_df, fhs_samples_df, on=["dbGaP_Sample_ID"])
print("fhs_expr_samples", fhs_expr_samples.shape)

# separate betweeh SABRe CVD (363) and SHARe (342)
gt_samples_from_fhs = fhs_samples_df[fhs_samples_df["STUDY"].str.contains("342")]
expr_samples_from_fhs = fhs_samples_df[fhs_samples_df["STUDY"].str.contains("363")]
print("expr_samples_from_fhs", expr_samples_from_fhs.shape)

FINAL_expr_df = pd.merge(expr_samples_df, expr_samples_from_fhs[expr_samples_from_fhs.SAMPLE_ID.str.contains("CEL")], on=["dbGaP_Sample_ID"])
print("FINAL_expr_df", FINAL_expr_df.shape)
print(list(FINAL_expr_df.columns))

# expr_samples_df (11329, 5)
# fhs_samples_df (64330, 8)
# fhs_expr_samples (22562, 12)
# expr_samples_from_fhs (11329, 8)
# FINAL_expr_df (5618, 12)

expr_samples_df (11329, 5)
fhs_samples_df (64330, 8)
fhs_expr_samples (22562, 12)
expr_samples_from_fhs (11329, 8)
FINAL_expr_df (5618, 12)
['dbGaP_Sample_ID', 'shareid', 'sampid', 'BODY_SITE', 'ANALYTE_TYPE', 'dbGaP_Subject_ID', 'BioSample Accession', 'SUBJECT_ID', 'SAMPLE_ID', 'SAMPLE_SOURCE', 'SOURCE_SAMPLE_ID', 'STUDY']


In [13]:
FINAL_expr_df

,dbGaP_Sample_ID,shareid,sampid,BODY_SITE,ANALYTE_TYPE,dbGaP_Subject_ID,BioSample Accession,SUBJECT_ID,SAMPLE_ID,SAMPLE_SOURCE,SOURCE_SAMPLE_ID,STUDY
0,805245,5,RWX110823_FHS_GENIII_49C09.CEL,blood,RNA,16960,SAMN01159697,5,RWX110823_FHS_GENIII_49C09.CEL,NaN,NaN,phs000363.v17.p11
1,706726,9,RWX100623_FHS_CCS_06H04.CEL,blood,RNA,16962,SAMN00968888,9,RWX100623_FHS_CCS_06H04.CEL,NaN,NaN,phs000363.v17.p11
2,803706,11,KAW110912_FHS_GENIII_52F12.CEL,blood,RNA,16963,SAMN01157619,11,KAW110912_FHS_GENIII_52F12.CEL,NaN,NaN,phs000363.v17.p11
3,805939,26,RWX111114_FHS_GENIII_63H03.CEL,blood,RNA,16969,SAMN01160208,26,RWX111114_FHS_GENIII_63H03.CEL,NaN,NaN,phs000363.v17.p11
4,707784,28,RWX110201_FHS_OFF_23G12.CEL,blood,RNA,16971,SAMN00969946,28,RWX110201_FHS_OFF_23G12.CEL,NaN,NaN,phs000363.v17.p11
5,706717,35,RWX100623_FHS_CCS_06F03.CEL,blood,RNA,16974,SAMN00968879,35,RWX100623_FHS_CCS_06F03.CEL,NaN,NaN,phs000363.v17.p11
6,707051,36,RWX101108_FHS_OFF_11H05.CEL,blood,RNA,16975,SAMN00969213,36,RWX101108_FHS_OFF_11H05.CEL,NaN,NaN,phs000363.v17.p11
7,707204,37,RWX101122_FHS_OFF_13E05.CEL,blood,RNA,16976,SAMN00969366,37,RWX101122_FHS_OFF_13E05.CEL,NaN,NaN,phs000363.v17.p11
8,707002,51,RWX101103_FHS_OFF_10E12.CEL,blood,RNA,16981,SAMN00969164,51,RWX101103_FHS_OFF_10E12.CEL,NaN,NaN,phs000363.v17.p11
9,706042,64,KAW110222_FHS_OFF_26D12.CEL,blood,RNA,16985,SAMN00968204,64,KAW110222_FHS_OFF_26D12.CEL,NaN,NaN,phs000363.v17.p11


In [14]:
FINAL_expr_ped_ages_df = pd.merge(FINAL_expr_df, ped_ages_df, on=["dbGaP_Subject_ID", "shareid"])
print(FINAL_expr_ped_ages_df.shape)

# Get sample ids for genotypes now
gt_sample_file = basedir+"genotypes/phg000679.v2_release_manifest.txt"
gt_df = pd.read_csv(gt_sample_file, comment="#", sep="\t", header=0)
gt_df = gt_df.drop(columns=["File_Name_Mtrx", "Tar_Name", "File_Name", "Status_Descript","Sample_Status","Sample_Use","IID", "Dataset", "Tar_Name_Mtrx", "File_Type", "Notes"])
print(gt_df.shape)
FINAL_PHENOTYPES = pd.merge(FINAL_expr_ped_ages_df, gt_df, left_on="shareid", right_on="Sample_ID")
print(FINAL_PHENOTYPES.shape)

FINAL_PHENOTYPES_expr = pd.merge(FINAL_PHENOTYPES, expr_data_df, left_on="sampid", right_on="cel_files")
print(FINAL_PHENOTYPES_expr.shape)
FINAL_PHENOTYPES_expr.head()


FINAL_PHENOTYPES_expr.to_csv(basedir+"compiled_annotations.txt", sep="\t", header=True, index=False)

(5443, 20)
(8453, 3)
(5101, 23)
(5101, 34)


In [17]:
FINAL_PHENOTYPES_expr.columns

Index(['dbGaP_Sample_ID', 'shareid', 'sampid', 'BODY_SITE', 'ANALYTE_TYPE',
       'dbGaP_Subject_ID', 'BioSample Accession', 'SUBJECT_ID', 'SAMPLE_ID',
       'SAMPLE_SOURCE', 'SOURCE_SAMPLE_ID', 'STUDY', 'pedno', 'fshare',
       'mshare', 'sex', 'twinid', 'idtype', 'birthyr', 'age', 'Sample_ID',
       'Subject_ID', 'Subject_Consent', 'cel_files', 'RevisedSabreID',
       'all_probeset_mean', 'all_probeset_stdev', 'neg_control_mean',
       'neg_control_stdev', 'pos_control_mean', 'pos_control_stdev',
       'all_probeset_rle_mean', 'all_probeset_mad_residual_mean', 'mm_mean'],
      dtype='object')

In [113]:
# Check how many of them we actually have genotypep
gt_actual_samples = pd.read_csv("/cbscratch/franco/datasets/FHS/genotypes/merged/chr10.fhs.dosages.sample", skiprows=2, sep=" ", header=None)
FINAL_PHENOTYPES_expr[FINAL_PHENOTYPES_expr.shareid.isin(gt_actual_samples[0])].shape

# Tada! all of them!


(5101, 34)

In [55]:
#################################
# We can include some lab data, as blood cell counts
#################################
labc1 = pd.read_csv(c1_labfile, comment="#", sep="\t", header=0)
labc2 = pd.read_csv(c2_labfile, comment="#", sep="\t", header=0)
lab_df = pd.concat([labc1, labc2])

# 'HGB', 'HCT', 'MCV', 'MCH', 'MCHC', 'RDW',
sel_cols = ["dbGaP_Subject_ID","shareid","IDTYPE", "WBC", 'RBC', "PLT", "NE_PER", "LY_PER", "MO_PER", "EO_PER", "BA_PER"] 
print("labfhs", lab_df.shape)

                                           
# # ped_ages_lab_df = pd.merge(ped_ages_df, lab_df[sel_cols], on="dbGaP_Subject_ID")
# # print(ped_ages_lab_df.shape) # 3388, 22

FINAL_PHENOTYPES_EXPR_LABFHS = pd.merge(FINAL_PHENOTYPES_expr, lab_df, on=["dbGaP_Subject_ID", "shareid"], how="left")
print(FINAL_PHENOTYPES_EXPR_LABFHS.shape)

# #################################
# ### Not required, tables already contain the latest ids
# #################################
# custom_remapping_file = basedir+"expression/Sample_remapping_revision.txt"
# remap_df = pd.read_csv(custom_remapping_file, sep="\t", header=None)
# remap_df.columns = ["sampid", "V2_SABReID", "V2_SubjectID", "empty", "V3_sampleID", "V3_SubjectID(shareID)", "V3_SABReID"]

# pd.merge(remap_df, expr_samples_c2_df, on="sampid")

labfhs (3773, 51)
(5101, 83)


In [56]:
print(np.sum(FINAL_PHENOTYPES_expr["sampid"].str.contains("GENIII")))
print(np.sum(FINAL_PHENOTYPES_expr["sampid"].str.contains("OFF")))
print(np.sum(FINAL_PHENOTYPES_expr["sampid"].str.contains("CCS")))
print(FINAL_PHENOTYPES_expr.shape)

2965
1692
377
(5101, 34)


In [57]:
print(np.sum(FINAL_PHENOTYPES_EXPR_LABFHS["sampid"].str.contains("GENIII")))
print(np.sum(FINAL_PHENOTYPES_EXPR_LABFHS["sampid"].str.contains("OFF")))
print(np.sum(FINAL_PHENOTYPES_EXPR_LABFHS["sampid"].str.contains("CCS")))
print(FINAL_PHENOTYPES_EXPR_LABFHS.shape)

2965
1692
377
(5101, 83)


In [59]:
FINAL_PHENOTYPES_EXPR_LABFHS.to_csv("Framingham_all_cohorts_ped_age_labdata.txt", sep="\t", header=True)

In [54]:
FINAL_PHENOTYPES_EXPR_LABFHS.columns

Index(['dbGaP_Sample_ID', 'shareid', 'sampid', 'BODY_SITE', 'ANALYTE_TYPE',
       'dbGaP_Subject_ID', 'BioSample Accession', 'SUBJECT_ID', 'SAMPLE_ID',
       'SAMPLE_SOURCE', 'SOURCE_SAMPLE_ID', 'STUDY', 'pedno', 'fshare',
       'mshare', 'sex', 'twinid', 'idtype', 'birthyr', 'age', 'Sample_ID',
       'Subject_ID', 'Subject_Consent', 'cel_files', 'RevisedSabreID',
       'all_probeset_mean', 'all_probeset_stdev', 'neg_control_mean',
       'neg_control_stdev', 'pos_control_mean', 'pos_control_stdev',
       'all_probeset_rle_mean', 'all_probeset_mad_residual_mean', 'mm_mean',
       'IDTYPE', 'UR_ALB', 'UR_CREAT', 'HBA1C', 'ALB', 'AST', 'ALT', 'BILI',
       'CA', 'GGT', 'PHOS', 'CHOL', 'TRIG', 'GLUC', 'GLUC2', 'CREAT', 'HDL',
       'CRP', 'WBC', 'RBC', 'HGB', 'HCT', 'MCV', 'MCH', 'MCHC', 'RDW', 'PLT',
       'MPV', 'NE_PER', 'LY_PER', 'MO_PER', 'EO_PER', 'BA_PER', 'NE_NUM',
       'LY_NUM', 'MO_NUM', 'EO_NUM', 'BA_NUM', 'Caffeine', 'hrs_fasting',
       'fasting', 'hdl_i', 'alt_i

In [98]:
##### Below is some code merging other tables, arriving to the same results
###########################################
# Slightly alternative way to get to the same results via
# the child study manifest
###########################################


expr_manifest_file=basedir+"expression/phe000002.v7_release_manifest.txt"
expr_manifest_df = pd.read_csv(expr_manifest_file, comment="#", sep="\t", header=0)
expr_manifest_df = expr_manifest_df.drop(columns=["File_Name_Mtrx", "Tar_Name", "File_Name", "Status_Descript","Sample_Status","Sample_Use","IID", "Dataset", "Tar_Name_Mtrx", "File_Type"])

# expr_data_df contains data about microarray quality measurements
# expr_manifest_df contains subject ids and microarray filenames
tmp = pd.merge(expr_manifest_df, expr_data_df, left_on="Sample_ID", right_on="cel_files")
print(tmp.shape)

# FINAL_expr_df contains 
tmp2 = pd.merge(FINAL_expr_df, tmp, left_on="shareid", right_on="Subject_ID")
print(tmp2.shape)

tmp3 = pd.merge(tmp2, ped_ages_df, on=["dbGaP_Subject_ID", "shareid"])
print(tmp3.shape)

tmp3.to_csv(basedir+"provisional_annotations.txt", sep="\t", header=True, index=False)

# tmp4 = pd.merge(tmp3, )

# Check how many of them we actually have genotypep
gt_actual_samples = pd.read_csv("/cbscratch/franco/datasets/FHS/genotypes/merged/chr10.fhs.dosages.sample", skiprows=2, sep=" ", header=None)
tmp3[tmp3.shareid.isin(gt_actual_samples[0])].shape

# meh, same shit

(5618, 14)
(5618, 26)
(5443, 34)


(5101, 34)

In [100]:
tmp3.groupby("age").count()

,dbGaP_Sample_ID,shareid,sampid,BODY_SITE,ANALYTE_TYPE,dbGaP_Subject_ID,BioSample Accession,SUBJECT_ID,SAMPLE_ID,SAMPLE_SOURCE,...,all_probeset_rle_mean,all_probeset_mad_residual_mean,mm_mean,pedno,fshare,mshare,sex,twinid,idtype,birthyr
age,,,,,,,,,,,,,,,,,,,,,
30,101,101,101,101,101,101,101,101,101,0,...,101,101,101,101,101,101,101,4,101,101
35,188,188,188,188,188,188,188,188,188,0,...,188,188,188,188,188,188,188,0,188,188
40,372,372,372,372,372,372,372,372,372,0,...,372,372,372,372,372,372,372,0,372,372
45,584,584,584,584,584,584,584,584,584,0,...,584,584,584,584,584,584,584,3,584,584
50,706,706,706,706,706,706,706,706,706,0,...,706,706,706,706,706,706,706,1,706,706
55,695,695,695,695,695,695,695,695,695,0,...,695,695,695,695,693,693,695,11,695,695
60,587,587,587,587,587,587,587,587,587,0,...,587,587,587,587,574,574,587,5,587,587
65,554,554,554,554,554,554,554,554,554,0,...,554,554,554,554,461,461,554,6,554,554
70,537,537,537,537,537,537,537,537,537,0,...,537,537,537,537,397,397,537,2,537,537
